In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from species import Plants, Animal

# Set matplot colors
color_list = ['#e6bc5e', '#c89254', '#8d907f', '#d78723', '#a37646', '#be651e', '#6b6250', '#8d5322', '#4b4728', '#2c2b1d', ]
# np.random.shuffle(color_list)
# color_list = color_list[::-1]
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=color_list)

## Run simulation

In [ ]:
from tqdm.notebook import tqdm

rodents = Animal(number=[3608, 3608], growth_stages=[2, 4, 12, 15], max_life_month=15, BR=7., unit_energy_scale=1, N_duration=3, r_scale=0.1)
foxes = Animal(number=[48, 48], growth_stages=[4, 8, 120, 150], max_life_month=150, BR=0.5, unit_energy_scale=10, N_duration=5, r_scale=0.01)

plant = Plants(1e5, 10, 15)
e_asm_fox = 0.2
e_asm_rodents = 0.1
steps = 1
simulation_years = 10

N_foxes = []
N_rodents = []
# Q_foxes = []
# Q_rodents = []
E_plants = []
E_rodents = []
E_foxes = []
for i in tqdm(range(365*simulation_years//steps)):
    N_f = np.sum(foxes.Q)
    N_r = np.sum(rodents.Q)
    E_p = np.sum(plant.E)
    E_f = np.sum(foxes.energy())
    E_r = np.sum(rodents.energy())
    
    if N_f <= 0 or N_r <= 0 or E_p <= 0:
        print(N_f, N_r, E_p, E_f, E_r)
        print("Crashed in %d days."%(i*steps))
        break
    
    N_foxes.append(N_f)
    N_rodents.append(N_r)
    E_plants.append(E_p)
    E_rodents.append(E_r)
    E_foxes.append(E_f)
    # Q_rodents.append(np.sum(rodents.Q, axis=1))
    # Q_foxes.append(np.sum(foxes.Q, axis=1))
    
    d_fox = np.sum(foxes.demand())
    e_rod = np.sum(rodents.energy())
    foxes.step(0, e_rod ** e_asm_fox, steps=steps)
    rodents.step(d_fox / e_asm_fox, plant.E * e_asm_rodents, steps=steps)
    plant.E -= e_rod / e_asm_rodents
    plant.step(steps)

x = np.arange(len(N_rodents))
plt.plot(x, N_rodents)
plt.plot(x, N_foxes)
plt.legend(['rodents', 'foxes'])
plt.xlabel('Simlution days')
plt.ylabel('Number of a species')
plt.yscale('log')
plt.show()
plt.figure()
plt.plot(x, E_rodents)
plt.plot(x, E_foxes)
plt.plot(x, E_plants)
plt.legend(['rodents', 'foxes', 'plants'])
plt.yscale('log')
plt.show()


#### Evolution curve

In [ ]:
plt.figure(dpi=200)

x = np.arange(len(N_rodents))/365
plt.plot(x, N_rodents)
plt.plot(x, N_foxes)
plt.legend(['rodents', 'foxes'])
plt.xlabel('Simulation duration / year')
plt.ylabel('Number of a species')

N_r_20 = N_rodents[3*365:]
N_f_20 = N_foxes[3*365:]
m_r = np.mean(N_r_20)
vr_r = np.std(N_r_20)
m_f = np.mean(N_f_20)
vr_f = np.std(N_f_20)

plt.axhline(y=m_r, c='#8d907f', ls='--')
plt.axhline(y=m_f, c='#8d907f', ls='--')
plt.annotate('$\hat{N}_{rodents}=%.1f \pm %.1f$'%(m_r, vr_r), xy=(np.mean(x)*1.2, m_r*0.8))
plt.annotate('$\hat{N}_{foxes}=%.1f \pm %.1f$'%(m_f, vr_f), xy=(np.mean(x)*1.2, m_f*0.8))

plt.title('Simulation result of Baseline.')
plt.grid()
plt.yscale('log')
plt.savefig('fig/baseline_100y.png')
print(m_f/m_r, vr_f/vr_r)
plt.show()



#### Water Fall Chart

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def waterfall(var, species, name='rodents', bars=20 ):
    fig = plt.figure(dpi=400)
    ax = Axes3D(fig)

    x = np.arange(bars)
    for i in range(bars):
        idx = i * len(var) // bars
        y = np.mean(var[idx:idx+len(var) // bars, :], axis=0)
        # y = np.log(Q_rodents[idx, :]+1) 
        y = np.resize(y, [bars])
        plt.bar(x, y, zs=i, zdir='y', alpha=0.7)
        scale = max(y)


    r = []
    for i in range(len(var)):
        Q = var[i, :]
        tmp = np.sum(Q * (species.infant_prob + species.juvenile_prob)) / np.sum(Q)
        r.append(tmp)
    x = np.ones(len(r)) * (bars)
    y = np.arange(len(r)) / len(r) * bars 
    z = np.array(r) / np.max(r) * scale * 1.5
    plt.plot(x, y, z)
    x = bars+1
    idx = np.argmax(z)
    y = idx * 1.0 / len(r) * bars
    z = z[idx]
    ax.text(x, y, z, '%.2f'%(np.max(r)))
    
    plt.xlabel('Age: 0→$a_{max}$ ')
    plt.xticks([])
    ax.invert_yaxis()
    plt.ylabel('10 year ← 0 year')
    plt.yticks(range(0, bars, 5))
    ax.set_zlabel('Number')
    ax.view_init(1, 120)
    plt.title('Evolution of %s.'%(name))
    
    plt.savefig('fig/waterfall_%s.png'%(name))
    plt.show()
    
Q_rodents = np.array(Q_rodents)
Q_foxes = np.array(Q_foxes)
waterfall(Q_foxes, foxes, name='foxes')

## Desiplay boudary conditions

In [ ]:
s = foxes

# 1.Grow
plt.figure(dpi=500)
plt.plot(s.infant_prob)
plt.plot(s.juvenile_prob)
plt.plot(s.adult_prob)
plt.plot(s.senior_prob)
plt.plot(s.senior_prob + s.infant_prob + s.juvenile_prob + s.adult_prob,'--')
plt.title('Age-stages ditributions')
plt.xlabel('Age / days')
plt.ylabel('Probability')
plt.legend(['$r_{infant}$', '$r_{juvenile}$', '$r_{adult}$', '$r_{senior}$', '$\sum$'])
plt.savefig('fig/Age-stages ditributions.png')
plt.show()

plt.figure(dpi=500)
plt.plot(s.BR_m)
plt.plot(s.E_m)
plt.plot(s.r_DA_m)
plt.plot(s.r_dem_m)
plt.title('Monthly ditributed conditions')
plt.xlabel('month in a year')
plt.ylabel('Value')
plt.legend(['$BR_{monthly}(m)$', '$w_{energy}{(m)$', '$r_{DA}(m)$', '$r_{demand}(m)$'])
plt.grid()
plt.savefig('fig/monthly ditributed boundaries.png')
plt.show()

plt.figure(dpi=500)
plt.plot(s.E_a)
plt.plot(s.r_DA_a)
plt.plot(s.r_dem_a)
plt.plot(s.P_alloc)
plt.title('Age-stages ditributed conditions')
plt.xlabel('Age / days')
plt.ylabel('Value')
plt.legend(['$E_{individual}(a)$', '$r_{DA}(a)$', '$r_{demand}(a)$', '$P_{alloc}(a)$'])
plt.yscale('log')
plt.grid()
plt.savefig('fig/Age-stages ditributed boundaries.png')
plt.show()

## Sensitivity

In [ ]:
def sensitivity(val=1e5, max_year=3):
    cnt = -1
    vals = [val*0.8, val, val*1.2]
    N_fs = np.zeros([365*max_year, len(vals)])
    N_rs = np.zeros([365*max_year, len(vals)])
    for val in vals:
        cnt += 1
        rodents = Animal(number=[1e4, 1e4], growth_stages=[2, 4, 12, 15], max_life_month=15, BR=7., unit_energy_scale=1, N_duration=3, r_scale=0.1)
        foxes = Animal(number=[1e2, 1e2], growth_stages=[4, 8, 120, 150], max_life_month=150, BR=0.5, unit_energy_scale=10, N_duration=5, r_scale=0.01)
        
        t1 = 10
        plant = Plants(val, t1, t1*1.5)
        e_asm_fox = 0.2
        e_asm_rodents = 0.1
        steps = 1

        N_foxes = []
        N_rodents = []
        E_plants = []
        E_rodents = []
        E_foxes = []
        for i in tqdm(range(365*max_year//steps)):
            N_f = np.sum(foxes.Q)
            N_r = np.sum(rodents.Q)
            E_p = np.sum(plant.E)
            E_f = np.sum(foxes.energy())
            E_r = np.sum(rodents.energy())

            if N_f <= 0 or N_r <= 0 or E_p <= 0:
                print(N_f, N_r, E_p, E_f, E_r)
                print("Crashed in %d days."%(i*steps))
                break

            N_foxes.append(N_f)
            N_rodents.append(N_r)
            E_plants.append(E_p)
            E_rodents.append(E_r)
            E_foxes.append(E_f)

            d_fox = np.sum(foxes.demand())
            e_rod = np.sum(rodents.energy())
            foxes.step(0, e_rod ** e_asm_fox, steps=steps)
            rodents.step(d_fox / e_asm_fox, plant.E * e_asm_rodents, steps=steps)
            plant.E -= e_rod / e_asm_rodents
            plant.step(steps)
        N_fs[:len(N_foxes), cnt] = N_foxes
        N_rs[:len(N_rodents), cnt] = N_rodents
    return N_fs, N_rs

N_fs, N_rs = sensitivity()

#### Display sensetivity result. 

In [ ]:
c1 = '#2c2b1d'
c1_= '#8d5322'
c2 = '#a37646'
c2_= '#d78723'

x = np.arange(N_fs.shape[0])
plt.plot(x, N_fs[:,1], color=c1)
plt.plot(x, N_rs[:,1], color=c2)
plt.plot(x, N_fs[:,0], '--', color=c1)
plt.plot(x, N_fs[:,2], '-.', color=c1)
plt.fill_between(x, N_fs[:,0], N_fs[:,1], color=c1, alpha=0.5)
plt.fill_between(x, N_fs[:,2], N_fs[:,1], color=c1, alpha=0.25)
plt.plot(x, N_rs[:,1], '--', color=c2)
plt.plot(x, N_rs[:,2], '-.', color=c2)
plt.fill_between(x, N_rs[:,0], N_rs[:,1], color=c2, alpha=0.5)
plt.fill_between(x, N_rs[:,2], N_rs[:,1], color=c2, alpha=0.25)
plt.yscale('log')
plt.legend(['rodents', 'foxes'])
plt.xlabel('Simulation days')
plt.ylabel('Number of a species')
plt.grid()
plt.title('$e_{foxes}$')
plt.savefig('fig/e_foxes.png')
plt.show()

## Hunting Demo

In [ ]:
import copy

def hunting_demo(r, f, p, scenario=0):
    e_asm_fox = 0.2
    e_asm_rodents = 0.1
    steps = 1
    simulation_years = 10

    # Hunt activity parameters.
    hunted = []
    Hunt_pattern = ['infant', 'juvenile', 'adult', 'senior']
    Hunt_month = np.ones([12])
    Hunt_desire_inayear = 10.

    if scenario == 1: # Overhunting
        Hunt_desire_inayear = 25
    elif scenario == 2: # Age prefered
        Hunt_pattern = ['infant']
    elif scenario == 3: # autumn
        Hunt_month = np.array([1, 1, 1, 1, 1, 1, 2, 5, 10, 10, 2, 1,])
    elif scenario == 4: # all
        Hunt_pattern = ['infant']
        Hunt_month = np.array([1, 1, 1, 1, 1, 1, 2, 5, 10, 10, 2, 1,])
        

    
    Hunt_month = Hunt_month / np.sum(Hunt_month) * Hunt_desire_inayear / 30

    N_foxes = []
    N_rodents = []
    E_plants = []
    E_rodents = []
    E_foxes = []
    for i in tqdm(range(365*simulation_years//steps)):
        N_f = np.sum(f.Q)
        N_r = np.sum(r.Q)
        E_p = np.sum(p.E)
        E_f = np.sum(f.energy())
        E_r = np.sum(r.energy())

        if N_f <= 5 or N_r <= 5 or E_p <= 5:
            print(N_f, N_r, E_p, E_f, E_r)
            print("Crashed in %d days."%(i*steps))
            break

        N_foxes.append(N_f)
        N_rodents.append(N_r)
        E_plants.append(E_p)
        E_rodents.append(E_r)
        E_foxes.append(E_f)

        R_EM = Hunt_month[f.date.month-1]
        hunted.append(R_EM)

        d_fox = np.sum(f.demand())
        e_rod = np.sum(r.energy())
        f.step(0, e_rod ** e_asm_fox, R_Migration=-R_EM, Migration_pattern=Hunt_pattern, steps=steps)
        r.step(d_fox / e_asm_fox, p.E * e_asm_rodents, steps=steps)
        p.E -= e_rod / e_asm_rodents
        p.step(steps)
    return N_rodents, N_foxes, hunted

In [ ]:
r = copy.deepcopy(rodents) 
f = copy.deepcopy(foxes)
p = copy.deepcopy(plant)
N_r_h, N_f_h, h_h = hunting_demo(r, f, p, scenario=0)

names = ['Hunt_desire_inayear', 'Avoid infants', 'Hunt in autumn', 'Paradigm']
plt.figure(dpi=200)

for i in range(4):
    r = copy.deepcopy(rodents) 
    f = copy.deepcopy(foxes)
    p = copy.deepcopy(plant)
    N_rodents, N_foxes, hunted = hunting_demo(r, f, p, scenario=i+1)
    
    # plt.subplot(2,2,i+1)
    plt.figure(dpi=200)
    plt.plot(N_r_h, '--')
    plt.plot(N_f_h, '--')
    plt.plot(N_rodents)
    plt.plot(N_foxes)
    
    plt.legend(['control rodents', 'control foxes', 'treatment rodents', 'treatment foxes'])
    plt.xlabel('Simlution days')
    plt.ylabel('Number of a species')
    plt.yscale('log')

    y = np.cumsum(hunted)
    plt.twinx()
    plt.plot(y, c='#8d5322')
    plt.plot(h_h, c='#8d5322')
    plt.legend(['hunted', 'control hunted'])
    plt.ylabel('Accumulation of hunted foxes')
    plt.ylim([0, 300])
    plt.title(names[i])
    print('Hunt result: %.1f'%np.sum(hunted))
    
    plt.savefig('fig/%s.png'%names[i])
    plt.show()